In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
url = "https://ca.milesplit.com/meets/643068-cif-state-xc-championships-2024/results/1108705/formatted/"
response = requests.get(url)
print(response.status_code)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
# Use selenium to capture the html of the results tables
# Because the tables seem to be loading dynamically only after the JavaScript executes
options = Options()
options.add_argument("--headless")
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options)
driver.get(url)
time.sleep(5)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
table = soup.find("table")
df = pd.read_html(str(table))[0]
print(df.head())
print(df.tail())
df.shape


In [ ]:
tables = soup.find_all("table")
print(len(tables))
dfs = []
for t, i in zip(tables, range(len(tables))):
    df = pd.read_html(str(t))[0]
    print("starting the ", i, " table")
    print(df.head())
    print(df.tail())
    print("shape: ", df.shape)
    dfs.append(df)


In [ ]:
# results are presented as alternating individual and team results
# split the dataframes into two lists of individual and team
individual_results = []
team_results = []
for n,i in zip(dfs, range(len(dfs))):
    if i%2 == 0:
        individual_results.append(n)
    else:
        team_results.append(n)

print(len(individual_results))
print(len(team_results))


In [ ]:
df = individual_results[0]
df.head()

In [ ]:
for table in individual_results:
    table.drop("Video", axis=1, inplace=True)

for table in individual_results:
    print(table.shape)
    print(table.head())

In [ ]:
for table in individual_results:
    table.rename(columns={table.columns[2]: "Grade"},inplace=True)
for table in individual_results:
    print(table.shape)
    print(table.head())

In [ ]:
# replace any missing grade values with the most common grade in that race
# and convert all grade columns to data type int

for table,i in zip(individual_results,range(10)):
    print("Starting table ", i)
    most_common_grade = table["Grade"].mode().iloc[0]
    print("most common: ", most_common_grade)
    na_values = table["Grade"].isna()
    for n,j in zip(na_values,range(len(na_values))):
        if n:
            table.loc[[j],"Grade"] = most_common_grade
    table["Grade"] = table["Grade"].astype(int)

In [ ]:
# check for missing values in other columns
for table in individual_results:
    print(table.isna().sum())

In [ ]:
# missing values in the Points column mean the athlete was competing individually
# can replace with 0 for now
for table in individual_results:
    table['Points'].fillna(0,inplace=True)
    table['Points'] = table['Points'].astype(int)

In [ ]:
# check for no duplicate athletes
for table,i in zip(individual_results,range(len(individual_results))):
    print("Looking for duplicates in table ", i)
    print(table[table["Athlete"].duplicated(keep=False)])

In [ ]:
# Right now the team column contains both the name of the school and the CIF section the school is from
# Split this into two columns to standardize with team results tables
for table in individual_results:
    table[["Team", "CIF_Section"]] = table["Team"].str.extract(r'^(.*) \((\w{2})\)$')

In [ ]:
individual_results[0].head()

In [ ]:
df = team_results[0]
df.head()

In [ ]:
for table in team_results:
    print(table.isna().sum())

In [ ]:
# check for no duplicate teams
for table,i in zip(team_results,range(len(team_results))):
    print("Looking for duplicates in table ", i)
    print(table[table["Team"].duplicated(keep=False)])

In [ ]:
for table in team_results:
    print(table.dtypes)

In [ ]:
# Right now the team contains both the name of the school and the CIF section the school is from
# Split this into two columns
for table in team_results:
    table[["Team", "CIF_Section"]] = table["Team"].str.extract(r'^(.*) \((\w{2})\)$')

In [ ]:
for table in team_results:
    print(table.head())

In [ ]:
def format_mm_ss(time): # some 1-5 splits were provided in m:s format eg 0:7, which would be better as 0:07
    minutes, seconds = str(time).split(":")
    return f"{int(minutes):01}:{int(seconds):02}"

# Separate the Times column in 1-5 Split and Team time avg
for team in team_results:
    team[['1-5_Split', 'Team_Time_Avg']] = team['Times'].str.extract(r'(\d{1,2}:\d{1,2})\s+1-5 Split\s*\|\s*(\d{1,2}:\d{2}(?:\.\d{1,2})?) Avg')
    team['1-5_Split'] = team['1-5_Split'].apply(format_mm_ss)

In [ ]:
for team in team_results:
    print(team.head())

In [ ]:
for team in team_results:
    team.drop('Times', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
divisions = ["B1","B2","B3","B4","B5","G1","G2","G3","G4","G5"] #B1= Boys Division 1, G1= Girls Division 1

# designate which race the results were from by adding a "Division" column
for individual_df,team_df,division in zip(individual_results,team_results,divisions):
    individual_df['Division'] = division
    team_df['Division'] = division
    print(individual_df.head())
    print(team_df.head())
    

In [ ]:
# combine all individual results into one df
combined_individual_results = pd.concat(individual_results)
print(combined_individual_results.shape)
print(combined_individual_results.head())
print(combined_individual_results.tail())

In [ ]:
# combine all team results into one df
combined_team_results = pd.concat(team_results)
print(combined_team_results.shape)
print(combined_team_results.head())
print(combined_team_results.tail())

In [ ]:
combined_individual_results.to_csv("cif-individual-results.csv")
combined_team_results.to_csv("cif-team-results.csv")